# Creating XGBoost model. 

This is based on how we trained 55 separate models for 55 separate wavelengths. 

Note that the difference, other than using XGBoost model, is that Decision Trees does **not** require normalizing data. Hence we can go as it is. 

And since our data is pre-cleaned, we also do not need to put a pipeline into it. 

It is a plus that Decision Trees (and hence XGBoost) works best when the features are a collection of categorical and numerical features, OR purely numerical features, which the latter is for ours. 

And it's a plus if the number of features is far less than the number of training samples. We can drop features as well later during training randomly. 

However, since we are not familiar with XGBoost, and we have lots of features, tuning it is something of a requirement due to inexperience. We would do bayesian optimization for ourselves. Even though wandb offers pre-configured and easily sent job, we could learn more by implementing ourselves plus I have no idea how to retrieve best parameters from Weights and Biases. 

Also note that there's a chance we might not use the whole dataset for hparams tuning if it takes too long. 

In [1]:
storage_name = "baseline_xgboost_pred_1.txt"

PROJECT_ID = "sunlit-analyst-309609"
%env GCLOUD_PROJECT = $PROJECT_ID
%load_ext google.cloud.bigquery

# !export GOOGLE_APPLICATION_CREDENTIALS="/workspace/ariel_ml_2021/sunlit-analyst-309609-77b8e2f94cb5.json"

env: GCLOUD_PROJECT=sunlit-analyst-309609


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tempfile

import ast 
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm

import xgboost as xgb 
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split

import tensorflow as tf 

from google.cloud import bigquery, bigquery_storage
LOCATION = "us"

Examples taken from https://github.com/fmfn/BayesianOptimization/blob/master/examples/sklearn_example.py and https://www.kdnuggets.com/2019/07/xgboost-random-forest-bayesian-optimisation.html

If you look at their examples you'll find that they only have the function maximize, nothing on minimize. This means if we use RMSE or something we would not get something useful. Hence, there are two ways that could be think of. One, implement the Ariel Score as we want to maximize that. Second, use "negative (root) mean squared error". This way, it could be maximize as well. 

After deciding, `neg_mean_squared_error` would be a good choice. 

Tuning hyperparameters for tree-based learners. Based on Datacamp course on Intro to XGBoost. 

- Learning rate (eta). 
- Gamma: min loss reduction to create new tree split. 
- Lambda: (int) L2 regularization
- Alpha: (int) L1 regularization
- max_depth: (positive intger) how deep can a tree grows. 
- subsample: (0, 1]. Fraction of total training set that can be used for any given boosting round. Low means little amount of training data used, but may lead to underfitting. High might mean overfitting. 
- colsample_bytree: (0, 1]. The fraction of **features** that it can be used (selected) from during any given boosting round. Large value means (almost) all features can be used to build a tree. Smaller is additional regularization by restricting number of features. Using all columns might result in overfitting. 

In [3]:
def xgb_cv(X, y, target="label", **kwargs):
    """
    XGBoost Regressor Cross Validation Function.

    Parameters: 
        :var dataset: (Pandas.DataFrame) A Pandas DataFrame of our used dataset. 
        :var target: (str) The column name of the target. Default to "label". 
        :var kwargs: (dict) A dictionary for the optimizer to pass in as params for XGBoost
                Regressor. 
    """
    kwargs["objective"] = "reg:squarederror"
    estimator = xgb.XGBRegressor(**kwargs)

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

    # Using 4-fold validation. 
    cval = cross_val_score(estimator, X, y, scoring="neg_mean_squared_error",
                            cv=3)

    return cval.mean()

In [4]:
def bayesian_optimization(dataset, target, parameters, n_iter=10, init_points=3):
    """
    Bayesian Optimization Algorithm. 

    Parameters:
        :var dataset: (Pandas.DataFrame) A Pandas DataFrame of our used dataset. 
        :var target: (str) The column name of the target. Default to "label".
        :var parameters: (Python Dict) The dictionary containing the parameters (or its range) to
                optimize on. 
        :var n_iter: (int) How many steps of Bayesian Optimization to go through. The more steps 
                the more likely to find a good maximum. 
        :var init_points: (int) How many steps of random exploration to perform. Random 
                exploration can help in diversifying the exploration space. 
        :var **kwargs: other BayesianOptimization.maximize() parameters. 
    """
    y = dataset.pop(target)
    X = dataset
    
    def our_crossval(**kwargs):
        """
        Wrapper function for cross validation. 

        This might requires us to ensure casting of integer is correct, values passed in are   
        correct, etc. This includes capping to (0, 1) range for learning rate. 
        """
        return xgb_cv(
            X, y, target,
            learning_rate = max(min(kwargs["learning_rate"], 0.999), 1e-4),
            reg_lambda = int(kwargs["reg_lambda"]),
            reg_alpha = int(kwargs["reg_alpha"]),
            gamma = int(kwargs["gamma"]),
            subsample = max(min(kwargs["subsample"], 0.999), 1e-3),
            colsample_bytree = max(min(kwargs["colsample_bytree"], 0.999), 1e-3),
            max_depth = 15,
            n_estimators = 25,
        )

    optimizer = BayesianOptimization(
        f = our_crossval,
        pbounds = parameters,
        verbose = 2
    )

    optimizer.maximize(n_iter=n_iter)

    return optimizer.max

As of now I haven't know if there can be fixed values passed in as parameters to the bounds, as the bayesian optimization official github page does not have examples on such, so we will continue as such. 

In [5]:
parameters = {
    "learning_rate": (1e-4, 0.3),
    "reg_lambda": (1, 100),
    "reg_alpha": (1, 100),
    "gamma": (1, 100),
    "subsample": (0.3, 0.999),
    "colsample_bytree": (1e-3, 0.999),
}

target = "label"

Set up bigquery reading. 

In [6]:
from google.oauth2 import service_account

try:
    bqclient = bigquery.Client()
    bqstorageclient = bigquery_storage.BigQueryReadClient()
except:
    print("Using backup credentials.")
    
    credentials = service_account.Credentials.from_service_account_file(
        "/workspace/ariel_ml_2021/sunlit-analyst-309609-77b8e2f94cb5.json",
        scopes = ["https://www.googleapis.com/auth/cloud-platform"],
    )

    bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)
    bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

Using backup credentials.


We would only try for one file here to see if it runs. Then we would integrate this into real work later on. 

In [22]:
# %%bigquery df --use_bqstorage_api
# SELECT * EXCEPT (AAAA, BB, CC)
# FROM `sunlit-analyst-309609.training_set.train_table_19` a
# LEFT JOIN (
#     SELECT *
#     FROM `sunlit-analyst-309609.training_set.noisy_train_extra_params`
# ) b
# ON a.AAAA = b.AAAA

In [7]:
!pwd

/workspace/ariel_ml_2021


In [ ]:
# for table_num in range(55):
#     query_string = f"""
#     SELECT * EXCEPT (AAAA, BB, CC)
#     FROM `sunlit-analyst-309609.training_set.train_table_{table_num}`
#     """
    
#     df = (
#         bqclient.query(query_string)
#         .result()
#         .to_dataframe(bqstorage_client=bqstorageclient)
#     )
    
#     print("Starting Bayesian Optimization")
    
#     returned_value = bayesian_optimization(df, target=target, parameters=parameters, n_iter=15)
    
#     with open("target.txt", "a") as f:
#         f.write(f"Table number {table_num}\n")
#         f.write(str(returned_value))
#         f.write("\n\n")
    
#     print("Finished with table ", table_num)

Starting Bayesian Optimization
|   iter    |  target   | colsam... |   gamma   | learni... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.001407 |  0.5944   |  32.32    |  0.2605   |  43.14    |  33.05    |  0.4445   |
|  2        | -0.001287 |  0.1299   |  38.92    |  0.2931   |  46.39    |  42.53    |  0.5619   |
|  3        | -0.002758 |  0.8408   |  66.74    |  0.09636  |  45.97    |  43.65    |  0.4746   |
|  4        | -0.000969 |  0.628    |  48.28    |  0.2966   |  60.45    |  17.04    |  0.8293   |
|  5        | -0.0208   |  0.6885   |  72.89    |  0.04575  |  2.863    |  5.548    |  0.3856   |
|  6        | -0.001054 |  0.5439   |  38.01    |  0.23     |  46.56    |  43.99    |  0.6716   |
|  7        | -0.2019   |  0.999    |  1.0      |  0.0001   |  100.0    |  31.88    |  0.999    |
|  8        | -0.2019   |  0.999    |  20.26    |  0.0001   |  1.0      |  100.0    |  

## Step 2:

Find the parameters that give the lowest score, and do one more time cross validation with that parameters (for those with either not that parameter or having the target neg mean squared error too high to be considered ideal). Then, also include the original values as a choice (that is, default value), and then cross validate and see the score. Finally, compare both score with the original. If it is better than use that score, if it is worse we will stay with the tuned score. 

Write this into a new .txt file **with only the training parameters**. This will allow for indexing using numpy arrays loadtxt method. We are going to read from that file and convert it with `ast.literal_evals` into dict and pass the values into `XGBRegressor`  for training. **Don't forget to make integer values that are supposed to be integer before passing in**. 

In [64]:
# Read files from target.txt

test_df = []

with open("target.txt") as f:
    test_df.append(f.readlines())

test_df = np.squeeze(np.array(test_df))

test_np = []
for ii in np.arange(1, len(test_df), 3):
    test_np.append(test_df[ii].strip("\n"))

test_np = np.array(test_np)

test_np = np.array([ast.literal_eval(test_np[ii]) for ii in np.arange(len(test_np))])

del test_df

In [68]:
for ii in test_np:
    assert type(ii) is dict

assert len(test_np) == 55

In [78]:
test_np[0]["target"]

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 23.4 µs


-0.0007101904942502508

In [69]:
# Preprocessing: what should be int should be int, what should be limit between 0 and 1 should do
# so, before input into tuning as kwargs. 

def preprocessing(test_np):

    best_target_score = -999
    memory_loc = 0

    for ii in range(len(test_np)):
        target = test_np[ii]["target"]
        params = test_np[ii]["params"]

        params["learning_rate"] = max(min(params["learning_rate"], 0.999), 1e-4)
        params["reg_lambda"] = int(params["reg_lambda"])
        params["reg_alpha"] = int(params["reg_alpha"])
        params["gamma"] = int(params["gamma"])
        params["subsample"] = max(min(params["subsample"], 0.999), 1e-3)
        params["colsample_bytree"] = max(min(params["colsample_bytree"], 0.999), 1e-3)
        params["max_depth"] = 15
        params["n_estimators"] = 25

        # check for 'best' target score: 
        if target > best_target_score:
            best_target_score = target
            memory_loc = ii  # Use as retrieving value

        test_np[ii]["params"] = params

    return test_np, best_target_score, memory_loc


test_np, best_target_score, memory_loc = preprocessing(test_np)

The cell below have not been updated and the for loop have not been taken out. Howver, basically, trying to run with the default parameter takes too long to return, hence it is abandoned. Even if it does better, the time it took does not worth training for. 

In [77]:
# Try best target score parameters, default parameters, and assigned parameters. 
# Then for the same parameter, try whether doubling n_estimators improves training.

for table_num in range(len(test_np)):
    query_string = f"""
    SELECT * EXCEPT (AAAA, BB, CC)
    FROM `sunlit-analyst-309609.training_set.train_table_{table_num}`
    """
    
    df = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )

    y = df.pop(target)
    X = df

    print(f"Starting Table {table_num}.\n")

    # Try first objective: We can skip best target score as we already computed it.
    kwargs = [test_np[table_num]["params"], test_np[memory_loc]["params"]]
    curr_best = test_np[table_num]["target"]
    best = 0

    print(0, " ", curr_best)

    for i in range(1, 2):
        curr_mean = xgb_cv(X, y, "label", **kwargs[i])

        if curr_mean > curr_best:
            best = i
            curr_best = curr_mean

        print(i, " ", curr_mean)

    kwargs = kwargs[best]

    # Second objective: does doubling n_estimators improve training? 
    kwargs["n_estimators"] = 50
    curr_mean = xgb_cv(X, y, "label", **kwargs)
    print(i + 1, " ", curr_mean)

    if curr_mean > curr_best:
        curr_best = curr_mean
    else:
        kwargs["n_estimators"] = 25

    # Write curr best to file: 
    with open("second_target.txt", "a") as f:
        f.write(f"Table number {table_num}\n")
        f.write(str(kwargs))
        f.write("\n\n")

    print("\nFinished with table ", table_num, "\n")

Starting Table 0.

0   -0.0007101904942502508
1   -0.00028060634411458045


KeyboardInterrupt: 

# Own testing before bayesian optimization to make sure it run. 

In [59]:
query_string = f"""
SELECT * EXCEPT (AAAA, BB, CC)
FROM `sunlit-analyst-309609.training_set.train_table_19`
"""

df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)

In [62]:
m = copy.deepcopy(df)

y = m.pop("label")
X = m

In [77]:
my_param = {
    "learning_rate": 1e-3,
    "max_depth": 25,
    "subsample": 0.2,
    "colsample_bytree": 0.3,
    "objective": "reg:squarederror",
    "n_estimators": 500
}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=15, max_depth=10, verbosity=2)

xg_reg.fit(X, y)

[12:09:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 0 pruned nodes, max_depth=7
[12:09:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 0 pruned nodes, max_depth=7
[12:09:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 170 extra nodes, 0 pruned nodes, max_depth=10
[12:09:49] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 460 extra nodes, 0 pruned nodes, max_depth=10
[12:10:17] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 686 extra nodes, 0 pruned nodes, max_depth=10
[12:10:48] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 842 extra nodes, 0 pruned nodes, max_depth=10
[12:11:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 988 extra nodes, 0 pruned nodes, max_depth=10
[12:11:57] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1098 extra nodes, 0 pruned nodes, max_depth=10
[12:12:30] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1248 extra no

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=15, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=2)

In [78]:
X_test, y_test = X, y

preds = xg_reg.predict(X_test)

In [82]:
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f"RMSE: {rmse}")

RMSE: 0.00866371649129658


# To be done

Add `n_estimators` as a tuning parameter to xgboost? 

In [83]:
preds

array([0.0251145 , 0.0338163 , 0.0292234 , ..., 0.04004366, 0.03397508,
       0.03676901], dtype=float32)

In [81]:
y_test

0         0.025124
1         0.025124
2         0.025124
3         0.025124
4         0.025124
            ...   
125595    0.021874
125596    0.021874
125597    0.021874
125598    0.021874
125599    0.021874
Name: label, Length: 125600, dtype: float64